<a href="https://colab.research.google.com/github/antoniobelotti/gpu_aco_tsp/blob/master/gpu_aco_tsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter to /tmp/pip-req-build-89ay73hn
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter /tmp/pip-req-build-89ay73hn
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=d63cb05cad06c7ccaeacda0cd6c597d29bc629b9626ed88445ffec16d49937d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-6emzxvv7/wheels/68/c9/44/586a1f4aeb1b51f699323c9ef4675bede773f0a54074c8dd2c
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


#Code

In [ ]:
!git clone https://github.com/mastqe/tsplib.git

Cloning into 'tsplib'...
remote: Enumerating objects: 124, done.
remote: Total 124 (delta 0), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (124/124), 2.02 MiB | 9.83 MiB/s, done.
Resolving deltas: 100% (6/6), done.


###Code to read TSPLIB instances





In [ ]:
%%writefile tsp_instance.h
#ifndef TSP_TSP_INSTANCE_H
#define TSP_TSP_INSTANCE_H

#define ASCII_SLASH 47
#define ASCII_0 48
#define ASCII_9 57

typedef struct node_t{
    int id;
    int x;
    int y;
} Node;

typedef struct TSPInstance {
    const int numOfNodes;
    Node *nodes;
    float *edgeCosts;
} TSPInstance;


TSPInstance tsp_instance_read(const char *filename);
void tsp_instance_free(TSPInstance *instance);

#endif //TSP_TSP_INSTANCE_H

Writing tsp_instance.h


In [ ]:
%%writefile tsp_instance.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

#include "tsp_instance.h"


TSPInstance tsp_instance_read(const char *filename) {
    printf("Reading tsplib instances from %s\n", filename);

    FILE *f = fopen(filename, "r");
    if (f == NULL) {
        printf("Error reading file!");
        exit(1);
    }

    unsigned long filename_len = strlen(filename);
    const char *s = filename + filename_len; // pointer to last char
    for (; *s != ASCII_SLASH; s--); // scan backwards until / is found
    for (; *s < ASCII_0 || *s > ASCII_9; s++); // scan forward until a digit is found

    int num_of_nodes = strtol(s, NULL, 10);

    char line_buff[64];
    Node *nodes = malloc(sizeof *nodes * num_of_nodes);

    while (fgets(line_buff, sizeof line_buff, f) != NULL &&
           strncmp(line_buff, "NODE_COORD_SECTION", strlen("NODE_COORD_SECTION")) != 0);

    while (fgets(line_buff, sizeof line_buff, f) != NULL && strncmp(line_buff, "EOF", 3) != 0) {

        int progr, x, y;

        char *buff_cursor;
        progr = strtol(line_buff, &buff_cursor, 10);
        x = strtol(buff_cursor, &buff_cursor, 10);
        y = strtol(buff_cursor, &buff_cursor, 10);

        // convert to 0-indexed
        progr -= 1;

        nodes[progr].id = progr;
        nodes[progr].x = x;
        nodes[progr].y = y;

    }

    fclose(f);

    TSPInstance instance = {.numOfNodes = num_of_nodes, .nodes = nodes};

    float *edge_cost = malloc(sizeof(int) * num_of_nodes * num_of_nodes);

    float deltaX, deltaY;
    Node n1, n2;
    float weight;
    for (int i = 0; i < num_of_nodes; i++) {
        for (int j = i; j < num_of_nodes; j++) {
            n1 = nodes[i];
            n2 = nodes[j];

            deltaX = (float) (n1.x - n2.x);
            deltaY = (float) (n1.y - n2.y);

            weight = floorf(sqrtf(powf(deltaX, 2) + powf(deltaY,2)));

            edge_cost[n1.id * num_of_nodes + n2.id] = weight;
            edge_cost[n2.id * num_of_nodes + n1.id] = weight;
        }
    }

    instance.edgeCosts = edge_cost;
    return instance;
}

void tsp_instance_free(TSPInstance *instance) {
    free(instance->nodes);
    free(instance->edgeCosts);
}

Writing tsp_instance.c


##Sequential implementation

In [ ]:
%%writefile sequential_main.c
#include <stdio.h>
#include <limits.h>
#include <stdbool.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include "tsp_instance.h"

#define FILENAME "/content/tsplib/eil76.tsp"
#define MAX_ITERATIONS (100000)
#define ALPHA (1.0f) // importance of pheromone trail
#define BETA (3.0f)  // importance of heuristic visibility
#define RHO (0.8f)
#define Q (1.0f)
#define PHEROMONE_LB (0.01f)
#define STAGNATION_THRESHOLD (MAX_ITERATIONS/5) // after how many iteration without improvement do we stop


float randf(float right) {
    return ((float) rand() / (float) RAND_MAX) * right;
}

float pheromone_initialization(float *pheromones, const float *edge_cost, int N) {
    /* BUILD greedy path */
    int path[N];
    float path_cost = 0;
    bool visited[N];

    int id_first_node = (int) random() % N;
    visited[id_first_node] = true;

    int current_node = id_first_node;
    for (int i = 1; i < N; i++) {
        int closest_node = -1;
        float closest_node_cost = (float) INT_MAX;

        for (int j = 0; j < N; ++j) {
            if (!visited[j] && current_node != j && edge_cost[current_node * N + j] < closest_node_cost) {
                closest_node_cost = edge_cost[current_node * N + j];
                closest_node = j;
            }
        }

        path[current_node] = closest_node;
        visited[closest_node] = true;
        current_node = closest_node;
        path_cost += closest_node_cost;
    }

    /* set pheromone to N/greedy path cost */
    for (int i = 0; i < N * N; ++i)
        pheromones[i] = (float) N / (float) path_cost;

    
}

void build_paths(int *ant_paths_mx, float *pheromone_trails, float *edge_costs, int N) {
    for (int i = 0; i < N * N; i++) ant_paths_mx[i] = -1;

    bool unvisited_nodes_mx[N * N];
    for (int i = 0; i < N * N; i++) unvisited_nodes_mx[i] = true;

    float edge_fitness_mx[N * N];
    for (int r = 0; r < N; ++r)
        for (int c = 0; c < N; ++c)
            edge_fitness_mx[r * N + c] =
                    r == c ? 0 : powf(pheromone_trails[r * N + c], ALPHA) / powf(edge_costs[r * N + c], BETA);

    // build every ant's path
    for (int ant_id = 0; ant_id < N; ++ant_id) {

        // select random starting node
        int id_first_node = (int) random() % N;
        unvisited_nodes_mx[ant_id * N + id_first_node] = false;

        int current_node_id = id_first_node;
        for (int visited_nodes = 1; visited_nodes < N; visited_nodes++) {
            float prefix_sum[N];
            for (int j = 0; j < N; j++) {
                bool mask = unvisited_nodes_mx[ant_id * N + j];
                float fitness = edge_fitness_mx[current_node_id * N + j];
                prefix_sum[j] = (j > 0 ? prefix_sum[j - 1] : 0.0f) + ((float) mask * fitness);
            }

            float random_number = randf(prefix_sum[N - 1]);
            for (int j = 0; j < N; ++j) {
                float ps_prev = j > 0 ? prefix_sum[j - 1] : 0.0f;
                if (random_number >= ps_prev && random_number < prefix_sum[j]) {
                    ant_paths_mx[ant_id * N + current_node_id] = j;
                    unvisited_nodes_mx[ant_id * N + j] = false;
                    current_node_id = j;
                    break;
                }
            }
        }

        ant_paths_mx[ant_id * N + current_node_id] = id_first_node;
    }
}

int update_paths_len(const int *ant_paths_mx, float *ant_paths_len, const float *edge_cost, int N) {
    // keep track of the ant id with the shortest path
    int best_path_ant_id = -1;
    float best_path = (float) INT_MAX;

    for (int ant_id = 0; ant_id < N; ++ant_id) {
        // calculate cost of ant(ant_id) path
        ant_paths_len[ant_id] = 0;
        int current_node = 0;
        int next_node = ant_paths_mx[ant_id * N + current_node];

        for (int i = 0; i < N; ++i) {
            ant_paths_len[ant_id] += edge_cost[current_node * N + next_node];
            current_node = next_node;
            next_node = ant_paths_mx[ant_id * N + current_node];
        }

        if (ant_paths_len[ant_id] < best_path) {
            best_path = ant_paths_len[ant_id];
            best_path_ant_id = ant_id;
        }
    }

    return best_path_ant_id;
}

void pheromone_evaporation(float *pheromones, int N) {
    for (int i = 0; i < N * N; ++i)
        pheromones[i] = fmaxf((1 - RHO) * pheromones[i], PHEROMONE_LB);
}

void pheromone_update(float *pheromones, const int *paths, const float *paths_len, int N) {
    for (int i = 0; i < N; ++i) {
        for (int j = i; j < N; ++j) {
            float previous_pheromone_value = pheromones[i * N + j];
            float addition = 0.0f;
            for (int ant_id = 0; ant_id < N; ++ant_id)
                // if edge (i,j) is in path of ant ant_id
                if (paths[ant_id * N + i] == j)
                    addition += Q / (float) paths_len[ant_id];


            pheromones[i * N + j] = fmaxf(previous_pheromone_value + addition, PHEROMONE_LB);
        }
    }
}

void solve_sequential(TSPInstance instance) {
    int N = instance.numOfNodes;
    float pheromone_mx[N * N];
    int paths_mx[N * N];
    float paths_len[N];
    int best_path[N];
    float best_path_len = (float) INT_MAX;

    pheromone_initialization(pheromone_mx, instance.edgeCosts, N);
    int stagnation_counter = 0;
    for (int iter = 0; iter < MAX_ITERATIONS && stagnation_counter < STAGNATION_THRESHOLD; ++iter, ++stagnation_counter) {
        printf("Generation %d of %d", iter + 1, MAX_ITERATIONS);
        fflush(stdout);
      
        build_paths(paths_mx, pheromone_mx, instance.edgeCosts, N);
        int current_iteration_best_ant = update_paths_len(paths_mx, paths_len, instance.edgeCosts, N);

        pheromone_evaporation(pheromone_mx, N);
        pheromone_update(pheromone_mx, paths_mx, paths_len, N);

        if (paths_len[current_iteration_best_ant] < best_path_len) {
            best_path_len = paths_len[current_iteration_best_ant];

            // save best path
            for (int l = 0; l < instance.numOfNodes; ++l)
                best_path[l] = paths_mx[current_iteration_best_ant * N + l];

            // there's an improvement reset stagnation counter
            stagnation_counter = 0;
        }

        printf("\r");
        fflush(stdout);
    }

    char *prefix = stagnation_counter == STAGNATION_THRESHOLD ? "Stopped for stagnation!\n" : "";
    printf("%sBest Path has len: %f\n", prefix, best_path_len);
    for (int j = 0; j < instance.numOfNodes; ++j) printf("%d,", best_path[j] + 1);
    printf("\n");
    fflush(stdout);
}

int main() {
    srand(time(NULL));
    TSPInstance instance = tsp_instance_read(FILENAME);
    solve_sequential(instance);
    tsp_instance_free(&instance);
    return 0;
}

Writing sequential_main.c


**compile sequential version**

In [ ]:
%%writefile CMakeLists.txt
cmake_minimum_required(VERSION 3.21)
project(aco_tsp_sequential C)

set(CMAKE_C_STANDARD 99)

add_executable(aco_tsp_sequential sequential_main.c tsp_instance.c tsp_instance.h)

target_link_libraries(${PROJECT_NAME} PUBLIC m)

Writing CMakeLists.txt


In [ ]:
!mkdir cmake_build_debug
!cd cmake_build_debug
!cmake ../content
!cmake --build .

-- The C compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /content
[ 33%] Building C object CMakeFiles/aco_tsp_sequential.dir/sequential_main.c.o
[ 66%] Building C object CMakeFiles/aco_tsp_sequential.dir/tsp_instance.c.o
[100%] Linking C executable aco_tsp_sequential
[100%] Built target aco_tsp_sequential


In [ ]:
!./aco_tsp_sequential

Reading tsplib instances from /content/tsplib/eil76.tsp
Stopped for stagnation!
Best Path has len: 659.000000
51,30,12,45,15,74,53,35,32,58,31,17,27,59,57,33,40,24,14,37,47,61,56,49,50,67,54,21,52,5,55,44,73,46,7,69,48,10,9,63,42,43,1,3,29,8,36,26,23,18,68,34,65,19,25,22,13,72,66,70,64,2,16,41,38,11,76,6,71,20,60,39,62,28,4,75,


##GPU implementation

In [ ]:
%%cuda --name hello.cu
#include <stdio.h>
#include <iostream>

using namespace std;

__global__ void helloFromGPU (void) {
    printf("Hello World from GPU!\n");
}

int main(void) {
    // # hello from GPU 
    cout << "Hello World from CPU!" << endl;
    cudaSetDevice(1);
    helloFromGPU <<<1, 10>>>();
    cudaDeviceSynchronize();
    return 0;
}

'File written in /content/src/hello.cu'

In [ ]:
%%shell
nvcc -arch=sm_37 src/hello.cu -o hello
./hello

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
Hello World from CPU!
